In [ ]:
import random
import numpy
import itertools
import math

from deap import base
from deap import creator
from deap import tools
from deap.algorithms import eaSimple

In [ ]:
INPUT_FILE = 'data1.txt'

In [ ]:
# Helper functions

# Create an iterator to iterate over another iterator in chunks
def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return itertools.zip_longest(*args, fillvalue=fillvalue)

In [ ]:
num_people = 0
num_tables = 0
num_seats_per_table = 0
num_preferred_together = 0
num_preferred_apart = 0
preferred_together = []
preferred_apart = []

In [ ]:
def parse_pairs_from_file(f, num_pairs):
    preferences = [[] for i in range(num_people)]
    
    for i in range(num_pairs):
        x, y = [int(x) for x in f.readline().split()]
        
        assert(x != y and x >= 0 and x < num_people and y >= 0 and y < num_people)
            
        if not y in preferences[x]:
            preferences[x].append(y)
            preferences[y].append(x)

    return preferences

def load_from_file():
    global num_people, num_tables, num_seats_per_table, preferred_together, preferred_apart
    
    with open(INPUT_FILE, 'r') as f:
        # skip header line
        f.readline()
        
        config = [int(x) for x in f.readline().split()]
        num_people = config[0]
        num_tables = config[1]
        num_seats_per_table = config[2]
        num_preferred_together = config[3]
        num_preferred_apart = config[4]
    
        # skip header line
        f.readline()
        
        preferred_together = parse_pairs_from_file(f, num_preferred_together)
            
        # skip header line
        f.readline()
            
        preferred_apart = parse_pairs_from_file(f, num_preferred_apart)
        
load_from_file()
print('preferred together: {}'.format(preferred_together))
print('preferred_apart: {}'.format(preferred_apart))


In [ ]:
num_empty_seats = num_tables * num_seats_per_table - num_people
permutation_population = list(range(num_people)) + [None] * num_empty_seats

permutation_population

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("permutation", random.sample, permutation_population, len(permutation_population))
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.permutation)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
def initIndividual(icls, content):
    return icls(content)

def initPopulation(pcls, ind_init, num_people, num_tables, num_seats_per_table, preferred_together, preferred_apart, pop_size):
    assigned = set()
    arrangement = [[] for i in range(num_tables)]
    
    groups = [set([person] + friends[:]) for person, friends in enumerate(preferred_together[:math.ceil(num_people/2)]) if friends]
    
    # Group people who should sit together
    merged = True
    while merged:
        merged = False
        for i, group in enumerate(groups):
            for j, other_group in enumerate(groups[i+1:]):
                if any(person in other_group for person in group):
                    # Merge groups
                    merged = True
                    groups[i] = group.union(other_group)
                    groups.pop(i+j+1)
                    break
            if merged:
                break
    
    groups = [list(x) for x in groups]
            
    # Split oversize groups into smaller groups
    for i, group in enumerate(groups):
        if len(group) > num_seats_per_table:
            num_sub_groups = math.ceil(len(group) / num_seats_per_table)
            sub_groups = []
            for j in range(num_sub_groups - 1):
                sub_groups.append(group[j*num_seats_per_table:j*num_seats_per_table+num_seats_per_table])
            sub_groups.append(group[(num_sub_groups-1)*num_seats_per_table:])
            if len(sub_groups[-1]) == 1:
                sub_groups[-1].append(sub_groups[-2].pop(0))
            groups[i] = sub_groups[0]
            groups.extend(sub_groups[1:])
    
    # Assign groups to tables
    if len(groups) <= num_tables:
        arrangement = groups
    else:
        groups.sort(reverse=True, key=len)
        print('sorted groups = ', groups)
        
        j = 1
        group = groups[0]
        for i in range(len(arrangement)):
            while (len(group) <= num_seats_per_table - len(arrangement[i])):
                arrangement[i].extend(group)
                group = groups[j]
                j += 1
        
        
    for table in arrangement:
        assigned = assigned.union(set(table))
        

    # Assign remaining people, respecting apart preferences
    for person in range(num_people):
        if person in assigned:
            continue
        for table in arrangement:
            if num_seats_per_table - len(table) >= 1:
                # If there are already people at the table, make sure the person can sit with them
                can_sit_together = True
                if len(table) > 0:
                    for other_person in table:
                        if other_person in preferred_apart[person]:
                            can_sit_together = False
                            break
                    if not can_sit_together:
                        break
                if can_sit_together:    
                    table.append(person)
                    assigned.add(person)
                    break
    
    # Assign remaining people
    for person in range(num_people):
        if person in assigned:
            continue
        for table in arrangement:
            if num_seats_per_table - len(table) >= 1:
                table.append(person)
                assigned.add(person)
                break
    
    # Make sure each table has num_seats_per_table seats by appending None where necessary
    for table in arrangement:
        if len(table) < num_seats_per_table:
            table.extend([None]*(num_seats_per_table - len(table)))
    
    # Turn 2D array into 1D array
    flat_arrangement = []
    for table in arrangement:
        flat_arrangement.extend(table)
    
    return pcls(ind_init(flat_arrangement) for i in range(pop_size))


toolbox.register("individual_guess", initIndividual, creator.Individual)
toolbox.register("population_guess", initPopulation, list, toolbox.individual_guess, num_people, num_tables, num_seats_per_table, preferred_together, preferred_apart)

## Fitness Function

The goal state is to have all guests assigned to tables such that the bride and groom’s seating requests are met (ie. who should sit at the same table and who should sit at different tables).

**Constraint 1**: Pairs of guests that the bride and groom would like to sit together should be seated at the same table.

**Constraint 2**: Pairs of guests that the bride and groom would like to sit apart should be seated at different tables.

**Constraint 3**: Anyone whom the bride and groom would like to sit with at least one other person, should not be at a table with only strangers (this is to prevent splitting a group unevenly such that one person ends up alone).

Let `A` be the number of violations to Constraint 1.

Let `B` be the number of violations to Constraint 2.

Let `C` be the number of violations to Constraint 3.

`h(n) = 0.3A + 0.5B + 0.2C`

We want to minimize h, the number of constraint violations (optimal is h = 0).

In [ ]:
def evaluate(num_seats_per_table, preferred_together, preferred_apart, individual):
    print('preferred together ', preferred_together)
    print('preferred apart ', preferred_apart)
    print('individual = ', individual)
    
    penalty = 0
    num_people = len(preferred_together)
    tables = grouper(individual, num_seats_per_table)
    A = 0
    B = 0
    C = 0
    
    for table in tables:
        for person in table:
            if person is None:
                continue
            
            # Check that person is not alone when they should be with at
            # least one other person
            should_not_be_alone = False
            is_alone = True
            for other_person in preferred_together[person]:
                should_not_be_alone = True
                if other_person in table:
                    is_alone = False
                    break
                    
            if should_not_be_alone and is_alone:
                C += 1
                
            # Only consider the people whose id is the lesser of the pair
            # so that each symmetrical preference is calculated only once
            for other_person in preferred_together[person]:
                if other_person < person:
                    continue
                if other_person not in table:
                    A += 1
                    
            for other_person in preferred_apart[person]:
                if other_person < person:
                    continue
                if other_person in table:
                    B += 1
            
            
    return 0.3*A + 0.5*B + 0.2*C,

toolbox.register("evaluate", evaluate, num_seats_per_table, preferred_together, preferred_apart)

In [ ]:
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=50)

In [ ]:
seed_population_size = 10
mate_probability = 0.3
mutate_probability = 0.2
number_generations = 200

def main():
    # pop = toolbox.population(n=seed_population_size)
    pop = toolbox.population_guess(seed_population_size)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    eaSimple(pop, toolbox, mate_probability, mutate_probability, number_generations, stats, halloffame=hof)

    return pop, stats, hof

main()

In [ ]:
evaluate(num_seats_per_table, preferred_together, preferred_apart, [13, 14, 3, 16, 8, 18, 1, 7, 6, 12, 5, 11, 0, 2, 17, 15, 4, 19, 10, 9])

In [ ]:
def test_evaluate():
    num_people = 5
    num_tables = 2
    num_seats_per_table = 3
    preferred_together = [
        [ 1, 2 ],
        [ 0 ],
        [ 0 ],
        [ ],
        [ ]
    ]
    preferred_apart = [
        [ ],
        [ 3 ],
        [ ],
        [ 1 ],
        [ ]
    ]
    
    # No violations
    individual = [0, 1, 2, 3, 4, None]
    assert(evaluate(num_seats_per_table, preferred_together, preferred_apart, individual) == (0,))
    
    # A=1, B=1, C=1
    individual = [0, 1, 3, 2, 4, None]
    assert(evaluate(num_seats_per_table, preferred_together, preferred_apart, individual) == (1,))

test_evaluate()